In [1]:
data = open("../inputs/day-16").read().splitlines()

In [2]:
import re

In [3]:
sizes = {}
distances = {}
neighbors = {}

In [4]:
for l in data:
    name, value, neigh = re.match("Valve (\w+) has .*=(\d+); .* valves? (.*)", l).groups([1,2,3])
    sizes[name] = int(value)
    neigh = neigh.split(', ')
    neighbors[name] = set(neigh)
    distances[(name, name)] = 0
    for n in neigh:
        distances[(name, n)] = distances[(n, name)] = 1

In [5]:
def bfs(s):
    visited = set(sizes.keys()) - {s}
    Q = [s]
    while len(Q) > 0:
        node = Q.pop(0)
        for n in neighbors[node]:
            if n in visited:
                visited.remove(n)
                dist = distances.get((s,n), 99999)
                dist = min(dist, distances[s, node] + 1)
                distances[(s,n)] = distances[(n,s)] = dist
                Q.append(n)

In [6]:
for x in sizes.keys():
    bfs(x)

In [7]:
best_pipes = [x for x,y in sorted([(k,v) for k,v in sizes.items() if v > 0], key=lambda x: x[1], reverse=True)]

In [8]:
assert len(sizes) ** 2 == len(distances)

In [9]:
max(distances.values())

15

Solving by exhaustive search:

In [15]:
from itertools import permutations as perm, combinations as comb
import random

In [16]:
table = {}

In [17]:
best_values = {}

In [21]:
def length(steps):
    arr = ['AA'] + list(steps)
    ans = 0
    for i in range(len(arr) - 1):
        ans += distances[(arr[i], arr[i+1])]
        ans += 1
    return ans

In [22]:
def value(steps):
    if steps in table:
        return table[steps]
    ans = 0
    t = 0
    arr = ['AA'] + list(steps)
    
    for i in range(len(arr) - 1):
        t += distances[(arr[i], arr[i+1])]
        t += 1
        ans += sizes[arr[i+1]] * (30 - t)
    
    table[steps] = ans
    return ans

In [23]:
def find_best_value(lst):
    if (ans:=frozenset(lst)) in best_values:
        return best_values[ans]
    
    score = 0
    for p in perm(lst):
        if length(p) >= 30:
            continue
        v = value(p)
        score = max(score, v)
    
    best_values[frozenset(lst)] = score
    return score

In [24]:
ans = 0
for l in range(4,9):
    print(l)
    opt = comb(best_pipes, l)
    for o in opt:
        score = find_best_value(o)
        if score == 0:
            continue
        if score > ans:
            ans = score

4
5
6
7
8


In [25]:
print(ans)

2320


## Me + Elephant

Memoization:

In [26]:
table = {}

In [27]:
best_values = {}

In [28]:
def value(steps):
    if steps in table:
        return table[steps]
    ans = 0
    t = 0
    arr = ['AA'] + list(steps)
    
    for i in range(len(arr) - 1):
        t += distances[(arr[i], arr[i+1])]
        t += 1
        ans += sizes[arr[i+1]] * (26 - t)
    
    table[steps] = ans
    return ans

In [29]:
def find_best_value(lst):
    if (ans:=frozenset(lst)) in best_values:
        return best_values[ans]
    
    score = 0
    for p in perm(lst):
        if length(p) >= 26:
            continue
        v = value(p)
        score = max(score, v)
    
    best_values[frozenset(lst)] = score
    return score

This takes about 3 mintues:

In [30]:
ans = 0
for l1, l2 in [(4,4),(4,5),(4,6),(4,7),(5,5),(5,6),(5,7),(6,6),(6,7),(7,7),(7,8)]:
    print(l1,l2)
    opt1 = comb(best_pipes, l1)
    for o1 in opt1:
        score_1 = find_best_value(o1)
        if score_1 == 0:
            continue

        new_best = list(set(best_pipes) - set(o1))
        opt2 = comb(new_best, l2)

        for o2 in opt2:
            score_2 = find_best_value(o2)
            if score_2 == 0:
                continue

            new = score_1 + score_2
            if new > ans:
                ans = new

4 4
4 5
4 6
4 7
5 5
5 6
5 7
6 6
6 7
7 7
7 8


In [31]:
print(ans)

2967
